In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Stanford-AIMI/CheXagent.git

Cloning into 'CheXagent'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 201 (delta 26), reused 36 (delta 15), pack-reused 144 (from 1)
Receiving objects: 100% (201/201), 21.28 MiB | 31.00 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [ ]:
!pip install transformers==4.40.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


In [ ]:
import io

import requests
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
# step 1: Setup constant
model_name = "StanfordAIMI/CheXagent-2-3b"
dtype = torch.bfloat16
device = "cuda"

# step 2: Load Processor and Model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
model = model.to(dtype)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: Fut

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CheXagentForCausalLM(
  (model): CheXagentModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerN

In [ ]:
# step 3: Inference
paths = ["https://prod-images-static.radiopaedia.org/images/2089213/f12063879a29e672f675977fabdc89_gallery.jpeg"]
query = tokenizer.from_list_format([*[{'image': path} for path in paths], {'text': "is there any issue"}])
conv = [{"from": "system", "value": "You are a helpful medical assistant."}, {"from": "human", "value": query}]
input_ids = tokenizer.apply_chat_template(conv, add_generation_prompt=True, return_tensors="pt")
output = model.generate(
    input_ids.to(device), do_sample=True, num_beams=10, temperature=0.4, top_p=1, use_cache=True,
    max_new_tokens=512
)[0]
response = tokenizer.decode(output[input_ids.size(1):-1])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(response)

No


In [ ]:
! unzip '/content/drive/MyDrive/685_Project/Slake_Chest_Images.zip'

Archive:  /content/drive/MyDrive/685_Project/Slake_Chest_Images.zip
   creating: Slake_Chest_Images/
   creating: Slake_Chest_Images/xmlab198/
   creating: Slake_Chest_Images/xmlab361/
   creating: Slake_Chest_Images/xmlab153/
   creating: Slake_Chest_Images/xmlab359/
   creating: Slake_Chest_Images/xmlab154/
   creating: Slake_Chest_Images/xmlab366/
   creating: Slake_Chest_Images/xmlab392/
   creating: Slake_Chest_Images/xmlab162/
   creating: Slake_Chest_Images/xmlab350/
   creating: Slake_Chest_Images/xmlab196/
   creating: Slake_Chest_Images/xmlab368/
   creating: Slake_Chest_Images/xmlab191/
   creating: Slake_Chest_Images/xmlab357/
   creating: Slake_Chest_Images/xmlab165/
   creating: Slake_Chest_Images/xmlab131/
   creating: Slake_Chest_Images/xmlab303/
   creating: Slake_Chest_Images/xmlab304/
   creating: Slake_Chest_Images/xmlab136/
   creating: Slake_Chest_Images/xmlab332/
   creating: Slake_Chest_Images/xmlab335/
   creating: Slake_Chest_Images/xmlab138/
   creating: Slak

In [ ]:
from __future__ import annotations
import pathlib
from typing import Callable, List
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
import requests
from io import BytesIO


# ---------- tiny helper to fetch local paths **or** URLs --------------------
def _open_image(path_or_url: str | pathlib.Path) -> Image.Image:
    p = str(path_or_url)
    if p.startswith(("http://", "https://")):
        resp = requests.get(p, timeout=15)
        resp.raise_for_status()
        return Image.open(BytesIO(resp.content)).convert("RGB")
    return Image.open(path_or_url).convert("RGB")


# ---------------------------------------------------------------------------
class CSVModelRunner:
    """
    Run *any* model on a dataframe and append its answer.

    Parameters
    ----------
    model_fn    : Callable[[str, str], str]
                  Your function that takes (image_path, query_text) and
                  returns a string answer.
    input_csv   : CSV with at least two columns *or* anything your
                  image_fn/query_fn can handle.
    output_csv  : Where to write the augmented CSV.
    image_fn    : row -> str  (optional transformation)
    query_fn    : row -> str  (optional transformation)
    image_col   : fallback column for image path (default 'image_path')
    query_col   : fallback column for text      (default 'query_text')
    batch_size  : 1 = no batching; if your model_fn accepts lists you
                  can vectorise by changing this class slightly.
    output_col  : name of the new column (default 'model_output')
    """

    def __init__(
        self,
        model_fn: Callable[[str, str], str],
        input_csv: str | pathlib.Path,
        output_csv: str | pathlib.Path,
        *,
        image_fn=None,
        query_fn=None,
        image_col: str = "image_path",
        query_col: str = "query_text",
        output_col: str = "model_output",
        batch_size: int = 1,
    ):
        self.model_fn = model_fn
        self.df = pd.read_csv(input_csv)
        self.output_csv = pathlib.Path(output_csv)
        self.output_col = output_col
        self.batch_size = max(1, batch_size)

        # fall‑back to columns if no transformation given
        if image_fn is None:
            if image_col not in self.df.columns:
                raise ValueError(f"Missing column '{image_col}'")
            image_fn = lambda row: getattr(row, image_col)   # noqa: E731
        if query_fn is None:
            if query_col not in self.df.columns:
                raise ValueError(f"Missing column '{query_col}'")
            query_fn = lambda row: getattr(row, query_col)   # noqa: E731

        self.image_fn = image_fn
        self.query_fn = query_fn

    # --------------------------------------------------------------------
    # def run(self):
    #     answers: List[str] = []
    #     iterator = tqdm(self.df.itertuples(index=False), total=len(self.df))

    #     # single‑row loop (simple & memory‑friendly)
    #     for row in iterator:
    #         img_path = str(self.image_fn(row))
    #         query = self.query_fn(row)

    #         # debug
    #         print(query, " | ", img_path)

    #         answer = self.model_fn(img_path, query)
    #         answers.append(answer)

    #     self.df[self.output_col] = answers
    #     self.output_csv.parent.mkdir(parents=True, exist_ok=True)
    #     self.df.to_csv(self.output_csv, index=False)
    #     print(f"✅  Saved ➜ {self.output_csv.resolve()}")


    # def run(self, save_every: int = 1, show_progress: bool = True):
    #     """
    #     Parameters
    #     ----------
    #     save_every : int
    #         Flush results to disk after this many rows (last chunk is flushed
    #         at the end even if < save_every).
    #     """
    #     buf_rows, buf_answers = [], []
    #     first_write = not self.output_csv.exists()

    #     iterator = (
    #         tqdm(self.df.itertuples(index=False), total=len(self.df))
    #         if show_progress else self.df.itertuples(index=False)
    #     )

    #     for idx, row in enumerate(iterator):
    #         img_path = str(self.image_fn(row))
    #         query    = self.query_fn(row)
    #         answer   = self.model_fn(img_path, query)

    #         # keep original row + new answer in a dict
    #         out_row = row._asdict()
    #         out_row[self.output_col] = answer
    #         buf_rows.append(out_row)

    #         # flush if buffer full
    #         if len(buf_rows) >= save_every:
    #             self._flush(buf_rows, first_write)
    #             first_write = False   # header written once
    #             buf_rows.clear()

    #     # flush remainder
    #     if buf_rows:
    #         self._flush(buf_rows, first_write)

    #     print(f"✅  Finished – predictions saved incrementally to "
    #           f"{self.output_csv.resolve()}")

    # # ------------------------------------------------------------------
    # def _flush(self, rows_buffer, first_write: bool):
    #     """Write a list[dict] to CSV (append mode)."""
    #     pd.DataFrame(rows_buffer).to_csv(
    #         self.output_csv,
    #         mode="w" if first_write else "a",
    #         header=first_write,
    #         index=False,
    #     )

    def run(self, save_every=5):
        buf, first = [], not self.output_csv.exists()
        for i, row in enumerate(tqdm(self.df.itertuples(index=False), total=len(self.df))):
            answer = safe_model_fn(str(self.image_fn(row)), self.query_fn(row))
            out    = row._asdict(); out[self.output_col] = answer
            buf.append(out)

            # flush every N rows
            if len(buf) >= save_every:
                self._flush(buf, first); buf.clear(); first = False
        if buf: self._flush(buf, first)

    def _flush(self, rows, header):
        pd.DataFrame(rows).to_csv(
            self.output_csv, mode="w" if header else "a",
            header=header, index=False
        )


# def llava_rad_fn(img_path: str, query: str) -> str:
#     conv = conv_templates[conv_mode].copy()
#     conv.append_message(conv.roles[0], query)
#     conv.append_message(conv.roles[1], None)
#     prompt = conv.get_prompt()
#     print(prompt)
#     image = load_image(image_file)
#     # image = _open_image(img_path)
#     image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0].half().unsqueeze(0).cuda()

#     input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

#     stopping_criteria = KeywordsStoppingCriteria(["</s>"], tokenizer, input_ids)

#     with torch.inference_mode():
#         output_ids = model.generate(
#             input_ids,
#             images=image_tensor,
#             do_sample=False,
#             temperature=0.0,
#             max_new_tokens=1024,
#             use_cache=True)

#     outputs = tokenizer.batch_decode(output_ids[:, input_ids.shape[1]:], skip_special_tokens=True)[0]
#     outputs = outputs.strip()
#     print(outputs)
#     print()
#     print("*" * 100)
#     print()

#     return outputs


def chexagent_fn(img_path: str, query: str) -> str:
    paths = [str(img_path)]
    query = tokenizer.from_list_format([*[{'image': path} for path in paths], {'text': query}])
    # print(paths, query)
    conv = [{"from": "system", "value": "You are a helpful medical assistant."}, {"from": "human", "value": query}]
    input_ids = tokenizer.apply_chat_template(conv, add_generation_prompt=True, return_tensors="pt")
    output = model.generate(
        input_ids.to(device), do_sample=True, num_beams=10, temperature=0.4, top_p=1, use_cache=True,
        max_new_tokens=512
    )[0]
    response = tokenizer.decode(output[input_ids.size(1):-1])

    print(response)
    print()
    print("*" * 100)
    print()

    return response



In [ ]:
# import gc, torch, re

# # --- helper that works on every PyTorch version ----------------------------
# def _is_oom_error(exc: BaseException) -> bool:
#     """Return True if *exc* looks like a CUDA‑OOM."""
#     if isinstance(exc, getattr(torch.cuda, "OutOfMemoryError", RuntimeError)):
#         return True
#     return isinstance(exc, RuntimeError) and bool(
#         re.search(r"CUDA.*out of memory", str(exc), re.I)
#     )

# # --- drop‑in safe‑inference wrapper ----------------------------------------
# def safe_inference(img_path: str, query: str) -> str:
#     """
#     Run your model once and aggressively free VRAM afterwards.
#     Replace the single line with *your* inference call.
#     """
#     try:
#         with torch.no_grad():                     # no grad buffers
#             answer = chexagent_fn(img_path, query)   # <-- your function
#         return answer
#     except BaseException as e:
#         if _is_oom_error(e):
#             print("⚠️  Caught CUDA OOM.  Flushing cache and re‑raising.")
#             torch.cuda.empty_cache()              # flush allocator cache
#         raise                                      # propagate
#     finally:
#         # Remove stray GPU tensors your code may have created
#         for obj_name in ("image_tensor", "input_ids", "output_ids"):
#             if obj_name in globals():
#                 del globals()[obj_name]
#         gc.collect()                               # free Python refs
#         torch.cuda.empty_cache()                   # free cached blocks :contentReference[oaicite:2]{index=2}
#         torch.cuda.ipc_collect()                   # free IPC-handled blocks :contentReference[oaicite:3]{index=3}

import gc, torch, re

def is_cuda_oom(err):
    oom_cls = getattr(torch.cuda, "OutOfMemoryError", RuntimeError)
    return isinstance(err, oom_cls) or re.search("out of memory", str(err), re.I)

def safe_model_fn(img_path, query_text):
    try:
        with torch.no_grad():
            return chexagent_fn(img_path, query_text)  # ← your inference
    finally:
        # 1️⃣ delete GPU tensors you created (if any live outside the FN)
        for v in ["image_tensor", "input_ids", "output_ids"]:
            if v in globals():
                del globals()[v]
        gc.collect()                      # 2️⃣ run Python GC
        torch.cuda.empty_cache()          # 3️⃣ release cached blocks :contentReference[oaicite:2]{index=2}
        torch.cuda.ipc_collect()          # 4️⃣ flush IPC remnants :contentReference[oaicite:3]{index=3}



In [ ]:
from pathlib import Path
import pandas as pd

# Paths
INPUT_CSV = "/content/drive/MyDrive/685_Project/Slake_Chest_Data_Results_Both.csv"
SLICE_CSV = "/content/drive/MyDrive/685_Project/Slake_from_idx299.csv"
OUTPUT_CSV = "/content/drive/MyDrive/chexagent_SLAKE_preds_rag_from299.csv"

# 1. Read & slice
df = pd.read_csv(INPUT_CSV)
df_slice = df.iloc[299:].reset_index(drop=True)   # rows 299,300,...
df_slice.to_csv(SLICE_CSV, index=False)


# RAD_ROOT = Path("/content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder")
SLAKE_ROOT = Path("/content/")

# def image_transform(row):
#     filename = Path(row.image_path.replace("\\", "/")).name
#     return str(RAD_ROOT / filename)

def image_transform(row) -> str:
    relative = row.img_name
    print('hi', relative)
    return str(SLAKE_ROOT / relative)


runner = CSVModelRunner(
    model_fn  = safe_model_fn,
    input_csv = SLICE_CSV,
    output_csv= OUTPUT_CSV,
    image_fn  = image_transform,
    query_fn  = lambda r: str(r.question) + "\n" + str(r.top_k_contents),
)
runner.run()

  0%|          | 0/300 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hi Slake_Chest_Images/xmlab295/source.jpg
No acute cardiopulmonary process

****************************************************************************************************



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hi Slake_Chest_Images/xmlab295/source.jpg


...

hi Slake_Chest_Images/xmlab336/source.jpg
Pneumonia

****************************************************************************************************



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hi Slake_Chest_Images/xmlab336/source.jpg


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


No pneumothorax detected.

****************************************************************************************************

hi Slake_Chest_Images/xmlab336/source.jpg
No significant findings.

****************************************************************************************************

